
#**Accessing stable diffusion model via notebook**

Tutorial used: https://towardsdatascience.com/stable-diffusion-using-hugging-face-501d8dbdd8

# Install and import libraries

In [1]:
!pip install -qq -U diffusers transformers

In [2]:
!pip install accelerate

In [3]:
!pip install shap

In [4]:
!pip install git+https://github.com/openai/CLIP.git
!pip install open_clip_torch
!pip install sentence_transformers


  Cloning https://github.com/openai/CLIP.git to /private/var/folders/71/v9k425r51tddm6v4ztbmjhy40000gn/T/pip-req-build-9vf1uzsa
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/71/v9k425r51tddm6v4ztbmjhy40000gn/T/pip-req-build-9vf1uzsa
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [5]:
!pip install opencv-python

In [41]:
import diffusers
import accelerate
import shap
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
#from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import KFold
#from classification_models.tfkeras import Classifiers
import open_clip
import cv2
from sentence_transformers import util
from PIL import Image
import itertools
import math

## Running Stable Diffusion — High-level pipeline

In [3]:
from diffusers import StableDiffusionPipeline

Notebook login

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
print(torch.cuda.is_available())


False


In [4]:
if torch.backends.mps.is_available():
    DEFAULT_DEVICE = "mps"
else:
    DEFAULT_DEVICE = "cpu"

In [5]:
pipe = StableDiffusionPipeline.from_pretrained('CompVis/stable-diffusion-v1-4').to('mps')


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

# Generate dataset

In [6]:
#prompts_list = ['a flower lamp', 'a lamp with flowers']

In [20]:
prompts_list = ['a flower lamp', 'a lamp with flowers', 'a dog wearing hat', 'a Spanish flag', 'books and coffee', 'Girls watching movies', "a cat's house",'a map without land', 'ancient egyptian airpods', 'astronauts at parties']

## Generate images from prompts

In [21]:
for prompt in prompts_list:
  # Pass the prompt in the pipeline and save it as {prompt}.jpg
  pipe(prompt).images[0].save('{}.jpg'.format(prompt))


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

## Generate subsets of each prompt

In [22]:
def calc_subsets(prompt, results, subset, index):
  # Add the current subset to the result list
    results.append(subset[:])

    # Generate subsets by recursively including and excluding elements
    for i in range(index, len(prompt)):
        # Include the current element in the subset
        subset.append(prompt[i])

        # Recursively generate subsets with the current element included
        calc_subsets(prompt, results, subset, i + 1)

        # Exclude the current element from the subset (backtracking)
        subset.pop()


def subsets(prompt):
    #split string into list of strings:
    prompt = prompt.split()
    subset = []
    results = []
    index = 0
    calc_subsets(prompt, results, subset, index)

    #exclude empty set
    results = results[1:]

    #join strings
    results = [' '.join(result) for result in results]
    return results


* Generate subsets of the first prompt: 'a flower lamp'

In [8]:
subsets(prompts_list[0])

['a', 'a flower', 'a flower lamp', 'a lamp', 'flower', 'flower lamp', 'lamp']

##  Generate subsets of all prompts

In [23]:
prompt_subsets_dict = {}

for prompt in prompts_list:
  prompt_subsets_dict[prompt] = subsets(prompt)

## Generate images of all sub-prompts

In [24]:
for subset_list in list(prompt_subsets_dict.values()):
  for subset in subset_list:
    pipe(subset).images[0].save('{}.jpg'.format(subset))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

* Save the images
* compute similarity of each image with original prompt

* What is the average contribution of each token?
* each token -> player
* original generated image -> profit
* Use shapley values to quantify each feature's impact

* The value $ϕ_i(v)$ is player i's average contribution across all player orderings

Question: does the ordering of tokens matter?


# Calculate similarity score of each subset

In [25]:
# image processing model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 896, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=896, out_features=896, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=896, out_features=3584, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3584, out_features=896, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((896,), eps=1e-05, elementwise_affine

## functions to calculate similar scores using CLIP

In [26]:
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(image1, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(image2, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score
#similarity Score: 76.77
#print(f"similarity Score: ", round(generateScore(image1, image2), 2))

## Calculate marginal contribution of each feature

In [131]:
#create dataframe containing unique orig image and subset combination in a row
df = pd.DataFrame(prompt_subsets_dict.items(), columns= ['orig', 'subset'])
df = df.explode('subset').reset_index()
df

,index,orig,subset
0,0,a flower lamp,a
1,0,a flower lamp,a flower
2,0,a flower lamp,a flower lamp
3,0,a flower lamp,a lamp
4,0,a flower lamp,flower
...,...,...,...
89,9,astronauts at parties,astronauts at parties
90,9,astronauts at parties,astronauts parties
91,9,astronauts at parties,at
92,9,astronauts at parties,at parties


In [327]:
#df_subsets = df.astype('string')
df_subsets = df.copy()

In [328]:
# generate similarity score using generateScore function

df_subsets['similarity_score'] = df_subsets.apply(lambda row: generateScore('{}.jpg'.format(row['subset']), '{}.jpg'.format(row['orig'])), axis=1)


In [330]:
df_subsets['subsets_split'] = df_subsets['subset'].apply(lambda x: x.split())
#df_subsets['s'ubsets_split'] = df_subsets['subsets_split'].apply(lambda x: set(x))

In [333]:
#sort subsets for ease of calling them later
df_subsets['subsets_sorted'] = df_subsets['subsets_split'].apply(lambda x: ' '.join(x))

In [334]:
df_subsets.head(30)

,index,orig,subset,similarity_score,subsets_split,subsets_sorted
0,0,a flower lamp,a,10.40,[a],a
1,0,a flower lamp,a flower,37.86,"[a, flower]",a flower
2,0,a flower lamp,a flower lamp,100.00,"[a, flower, lamp]",a flower lamp
3,0,a flower lamp,a lamp,52.12,"[a, lamp]",a lamp
4,0,a flower lamp,flower,44.61,[flower],flower
5,0,a flower lamp,flower lamp,81.98,"[flower, lamp]",flower lamp
6,0,a flower lamp,lamp,60.16,[lamp],lamp
7,1,a lamp with flowers,a,23.15,[a],a
8,1,a lamp with flowers,a lamp,61.31,"[a, lamp]",a lamp
9,1,a lamp with flowers,a lamp with,66.79,"[a, lamp, with]",a lamp with


In [395]:
#create a dictionary of subset to similarity score
score_dict = pd.Series(df_subsets.similarity_score.values,index=df_subsets.subsets_sorted).to_dict()

## Calculate the Shapley Values
Now that we have our power set of models, we can calculate the Shapley values as the weighted average of the marginal contributions for each feature

* **Calculate marginal impact of each token**

In [336]:
df_subsets

,index,orig,subset,similarity_score,subsets_split,subsets_sorted
0,0,a flower lamp,a,10.40,[a],a
1,0,a flower lamp,a flower,37.86,"[a, flower]",a flower
2,0,a flower lamp,a flower lamp,100.00,"[a, flower, lamp]",a flower lamp
3,0,a flower lamp,a lamp,52.12,"[a, lamp]",a lamp
4,0,a flower lamp,flower,44.61,[flower],flower
...,...,...,...,...,...,...
89,9,astronauts at parties,astronauts at parties,100.00,"[astronauts, at, parties]",astronauts at parties
90,9,astronauts at parties,astronauts parties,75.82,"[astronauts, parties]",astronauts parties
91,9,astronauts at parties,at,38.56,[at],at
92,9,astronauts at parties,at parties,29.35,"[at, parties]",at parties


### function to get $S \cup i $

In [316]:
#how to get S_u_i

def get_S_u_i(S, i):
    S_split = S.split()
    #insert
    S_split.append(i)
    #sort
    S_split.sort()
    return ' '.join(S_split)

# function to calculate SHAP Values

In [403]:
#create dict to hold shap values of each token

dict_shap = {}

In [404]:
#calculate marginal contributions of each token i in prompt 'orig'
for orig in prompts_list:

    #create temporary dataframe for that prompt
    temp1 = df_subsets[df_subsets['orig']==orig].copy(deep=True)


    #Get list of original tokens
    tokens = orig.split()

    #create array to hold marginal contributions of tokens
    marginal_contributions = []

    #calculate marginal contribution of a token i using subsets not containing it (S)
    for i in tokens:

        #get rows of subsets of prompt not containing token i
        N = temp1[-temp1['subsets_split'].apply(lambda x: any(item ==i for item in x))]

        #get subsets  of prompt not containing token i
        S_array = np.array(N['subset'])

        #get length of subset
        S_size = len(S_array)

        marg_contr = 0

        #iterate through subsets of prompt not containing token i
        for S in S_array: 

            #sort S
            S = ' '.join(sorted(S.split()))

            #get value of S
            v_S = score_dict[S]

            #get value of S union i
            S_u_i = get_S_u_i(S, i)
            v_S_u_i = score_dict[S_u_i]


             #calc coalition prob
            S_size = len(S.split())
            n = len(tokens)
            coalition_prob = math.factorial(S_size) * math.factorial(n-S_size-1)/math.factorial (n)

            #add marginal contribution
            marg_contr += coalition_prob * (v_S_u_i - v_S)

        marginal_contributions.append(marg_contr)
        
    #divide by total to get SHAP values
    marginal_contributions = np.round(np.array(marginal_contributions)/sum(marginal_contributions),2)
    dict_shap[orig] = marginal_contributions
    

In [405]:
 dict_shap

{'a flower lamp': array([0.12, 0.35, 0.53]),
 'a lamp with flowers': array([0.21, 0.4 , 0.21, 0.18]),
 'a dog wearing hat': array([0.11, 0.4 , 0.23, 0.26]),
 'a Spanish flag': array([0.26, 0.32, 0.42]),
 'books and coffee': array([0.47, 0.11, 0.42]),
 'Girls watching movies': array([0.36, 0.19, 0.45]),
 "a cat's house": array([0.13, 0.59, 0.28]),
 'a map without land': array([0.04, 0.68, 0.2 , 0.08]),
 'ancient egyptian airpods': array([0.46, 0.38, 0.16]),
 'astronauts at parties': array([0.77, 0.18, 0.05])}

In [408]:
#convert dictionary to pandas dataframe

pd.DataFrame.from_dict(dict_shap, orient='index')

,0,1,2,3
a flower lamp,0.12,0.35,0.53,NaN
a lamp with flowers,0.21,0.40,0.21,0.18
a dog wearing hat,0.11,0.40,0.23,0.26
a Spanish flag,0.26,0.32,0.42,NaN
books and coffee,0.47,0.11,0.42,NaN
Girls watching movies,0.36,0.19,0.45,NaN
a cat's house,0.13,0.59,0.28,NaN
a map without land,0.04,0.68,0.20,0.08
ancient egyptian airpods,0.46,0.38,0.16,NaN
astronauts at parties,0.77,0.18,0.05,NaN


## Further questions:

(1) How do we account for the randomness in the images that are generated?

(2) Do the similarity values satisfy the superadditivity property? $v(s \cup t) \geq v(S) + v(T)$